In [7]:
import cv2 
import mediapipe as mp
import pyautogui
import numpy as np

In [8]:
# Initialize hand detection 
mp_hands = mp.solutions.hands.Hands(max_num_hands=1, min_detection_confidence=0.7, min_tracking_confidence=0.5, model_complexity=1, static_image_mode=False)
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

pyautogui.FAILSAFE = False
click_threshold = 0.1
alpha = 0.5

# define hand landmark to track
wrist_landmark_id = mp.solutions.hands.HandLandmark.WRIST
index_landmark_id = mp.solutions.hands.HandLandmark.INDEX_FINGER_TIP
thumb_landmark_id = mp.solutions.hands.HandLandmark.THUMB_TIP
middle_landmark_id = mp.solutions.hands.HandLandmark.MIDDLE_FINGER_TIP

def is_left_click(landmarks):
    index_tip = landmarks.landmark[index_landmark_id]
    thumb_tip = landmarks.landmark[thumb_landmark_id]
    distance = np.sqrt((index_tip.x - thumb_tip.x) ** 2 + (index_tip.y - thumb_tip.y) ** 2)
    return distance <= click_threshold

def is_right_click(landmarks):
    middle_tip = landmarks.landmark[middle_landmark_id]
    thumb_tip = landmarks.landmark[thumb_landmark_id]
    distance = np.sqrt((middle_tip.x - thumb_tip.x) ** 2 + (middle_tip.y - thumb_tip.y) ** 2)
    return distance <= click_threshold

def rotate_screen(landmarks):
    index_tip = landmarks.landmark[index_landmark_id]
    wrist = landmarks.landmark[wrist_landmark_id]
    # print(f"x: {(index_tip.y - index_tip.x)}")
    return (index_tip.x - wrist.x) < 0


# get screen size
screen_width, screen_height = pyautogui.size()

# sensitivity factor for cursor movement (adjust for desired speed)
sensivity = 0.5



# Initialize previous landmark positions
prev_fingertip_x = 0
prev_fingertip_y = 0

def mouse_position(landmarks):
    # index = 0.041
    # middle = 0.26
    # middlew = 0.1
    indexw = 0
    index_tip = landmarks.landmark[index_landmark_id]
    # thumb_tip = landmarks.landmark[thumb_landmark_id]
    middle_tip = landmarks.landmark[middle_landmark_id]
    # ring_tip = landmarks.landmark[mp.solutions.hands.HandLandmark.RING_FINGER_TIP]
    # pinky_tip = landmarks.landmark[mp.solutions.hands.HandLandmark.PINKY_TIP]
    
    wrist = landmarks.landmark[wrist_landmark_id]
    index_mcp = landmarks.landmark[mp.solutions.hands.HandLandmark.INDEX_FINGER_MCP]
    middle_mcp = landmarks.landmark[mp.solutions.hands.HandLandmark.MIDDLE_FINGER_MCP]
    index_pip = landmarks.landmark[mp.solutions.hands.HandLandmark.INDEX_FINGER_PIP]
    middle_pip = landmarks.landmark[mp.solutions.hands.HandLandmark.MIDDLE_FINGER_PIP]
    index_dip = landmarks.landmark[mp.solutions.hands.HandLandmark.INDEX_FINGER_DIP]
    middle_dip = landmarks.landmark[mp.solutions.hands.HandLandmark.MIDDLE_FINGER_DIP]
    # print(f"TIP: {index_tip.x} MCP: {index_mcp.x} PIP: {index_pip.x} DIP: {index_dip.x}")
    print(index_tip.x - wrist.x)


def cursor_movement(landmarks):
    index_tip = landmarks.landmark[index_landmark_id]
    movement_y = int(screen_height*index_tip.y)
    movement_x = int(screen_width*index_tip.x)
    
    pyautogui.moveTo(movement_x, movement_y)
    
def screenshot(landmarks):
    index_tip = landmarks.landmark[index_landmark_id]
    wrist = landmarks.landmark[wrist_landmark_id]
    middle_tip = landmarks.landmark[middle_landmark_id]
    index = 0.61
    middle = 0
    
    return (index_tip.x - wrist.x) < 0.61 and (middle_tip.x - wrist.x) < 0
    
    
    
    


In [9]:

        
def detect_hands(frame):
  """
  Detects hands in a frame using MediaPipe.

  Args:
      frame: The frame to be processed.

  Returns:
      A list of hand landmarks if a hand is detected, otherwise None.
  """
  rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB for MediaPipe
  results = mp_hands.process(rgb_frame)
  if results.multi_hand_landmarks:
    hand_landmarks = results.multi_hand_landmarks[0]  # Assuming only one hand
    return hand_landmarks
  else:
    return None
  
wrist_prev = None
index_tip_prev = None
thumb_tip_prev = None

while True:
  # Take a picture with the camera
  success, frame = cap.read()

  # 2. Spot the Tips (continued) - Find the fingertip in this picture
  hand_landmarks = detect_hands(frame)  # This function is defined later
  
  # flip the frame 
  frame = cv2.flip(frame, 1)

  # If we found a fingertip, let's move the cursor!
  if hand_landmarks:
    # Remember where the fingertip is now
    # fingertip_landmark = hand_landmarks.landmark[mp.solutions.hands.HandLandmark.INDEX_FINGER_TIP]
    mp_drawing.draw_landmarks(frame, hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS)
    
    cursor_movement(hand_landmarks)
    
    if screenshot(landmarks=hand_landmarks):
      shot = pyautogui.screenshot()
      shot.save(r'E:\\New folder\\CodeDump\\Python New Tut\\Machine Learning\\VirtualMouse\\screenshot.png')
    
  # Show the picture with the fingertip (optional)
  cv2.imshow('Hand', frame)

  # Close the program if you press 'Esc'
  if cv2.waitKey(5) & 0xFF == 27:
    break

# Clean up the camera
cap.release()
cv2.destroyAllWindows()



c:\Users\adity\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
